In [ ]:
import pandas as pd
import datasets

# helyi CSV fájl
input_data_file = ''

# CSV három oszlopal:
#    * question -- az elvárt felhasználói bemenet
#    * response_j -- a model elvárt válasza
#    * response_k -- példa egy rossz model válaszra
df = pd.readc_csv(input_data_file)

# HuggingFace dataset
ds = datasets.Dataset.from_pandas(df)

# dataset feltőltése s3-ba
s3_training_input_path = f's3://....'
ds.save_to_disk(s3_training_input_path)

In [ ]:
from sagemaker.pytorch import PyTorch

hyperparameters = {
    'aws_region': '...',
    'huggingface_token': '...',
    'dataset_path': s3_training_input_path
}

pytorch_estimator = Pytorch(
    source_dir="./src",
    entry_point="launch.py",
    train_instance_type="ml.g5.12xlarge",
    train_instance_count=1,
    framework_version="2.2.0",
    py_version="py310",
    hyperparameters=hyperparameters
    output_path="s3://..."
    # user settings
    role=role,
    subnets=["..."],
    security_group_ids=["..."],
    tags=[{"Key": "...", "Value": "..."}]
)

In [ ]:
s3_training_data_path = "s3://..."
pytorch_estimator.fit(s3_training_data_path)